In [1]:
import pandas as pd

In [2]:
#NOTE: This will not run without the walkability.csv file which is not in this repo because it's too large to upload to github. 

file = 'Resources/walkability.csv'
df = pd.read_csv(file, encoding="ISO-8859-1", low_memory=False)

In [4]:
#ADD LEADING ZEROS IN FRONT OF STATE AND COUNTY TO MATCH WITH THE SCHOOL DATA 'CNTY' KEY
df['STATEFP'] = df['STATEFP'].astype(str).apply('{:0>2}'.format)
df['COUNTYFP'] = df['COUNTYFP'].astype(str).apply('{:0>3}'.format)

#CONCATENATING 'STATEFP' WITH 'COUNTYFP' AND CREATING 'CNTY' COLUMN AS A PRIMARY KEY
df['CNTY'] = df['STATEFP'] + df['COUNTYFP']

In [5]:
#Break apart this large dataset into dataframes that will could be used to populate map visuals.  
population_df = df[['STATEFP','COUNTYFP', 'TotPop', 'HH']]
walkability_df = df[['STATEFP','COUNTYFP','NatWalkInd']]

In [6]:
#rename the columns to better understand the data. See Resources/documentation_guide.pdf for more context. 

# ----- population_df ----- # 
    #population: is the population of each area. The total sum represents on total country population (pre 2022 census)
    #households: How many homes/units in each area. 

# ----- walkability_df ----- # 
#walkability_score:
    #walkability score = (w/3) + (x/3) + (y/6) + (z/6)
#        Where w = CBG ranked score for intersection density
#                    x = CBG ranked score for proximity to transit stops
#                    y = CBG ranked score for employment mix
#                    z = CBG ranked score for employment and household mix 

population_df = population_df.rename(columns={'STATEFP':'state','COUNTYFP':'county', 'TotPop':'population', 'HH':'households'})
walkability_df = walkability_df.rename(columns={'STATEFP':'state','COUNTYFP':'county','NatWalkInd':'walkability_score'})

In [7]:
#Grouping data from area into county that will be used for county data analysis and visualizations. Rounding and 
#Reseting the index to allow combining the state and county code in the next step.

county_population_df = population_df.groupby(['state','county']).sum().round().reset_index()
county_walkability_df = walkability_df.groupby(['state','county']).mean().reset_index()

In [8]:
#Adding a unique_id that combines the state and county code. This could be used as a unique key because county codes are not unique. 

county_population_df['unique_id'] = county_population_df['state']+county_population_df['county']
county_walkability_df['unique_id'] = county_walkability_df['state']+county_walkability_df['county']

In [9]:
#Exporting dataframes to CSV files. This is needed because the Walkability.csv file is too large on it's own to upload onto Github. 

county_population_df.to_csv('Resources/county_population_df.csv')
county_walkability_df.to_csv('Resources/county_walkability_df.csv')

Adding 